# **FLUX UPSCALER FOR IMAGES & VIDEOS**
- **This notebook was implemented based on a ComfyUI Flux Upscale Workflow using the UltimateSDUpscale node.**
- You can use the free T4 GPU to run this notebook, but it is much slower than others.
-You can tick the `download_face_upscalers` checkbox in the 'Setup Environment' section before running it if you just want to upscale a face. You can then select 4xFaceUpSharpDAT.pth or 4xFaceUpSharpLDAT.pth as your upscale model.
-You can tick the `download_loRA` checkbox in the 'Setup Environment' section before running it if you want to use a loRA from either huggingface or civitai. Remember to input your civitai token if your download link is from civitai. Then tick the `use_loRA` checkbox in the loRA settings before running the upscaling.
- Adding prompts is optional, but it might produce better results.
- The number of steps to use depends on the denoise value and your image or video. The more blurry the images or videos, the higher the steps and denoise values you should use. I usually make the steps 20 times the denoise value. But note that the higher the denoise value, the more different from the original your images and videos will appear. You can experiment with other settings to find out what works best for you.
- In the video settings, you can choose to upscale every 2nd or 3rd frame from your video to speed up the process, then use the frame interpolation notebook, which is far faster than upscaling, to recover your frames and fps.

In [ ]:
# @title Setup Environment
# !pip install --quiet torch torchvision --index-url https://download.pytorch.org/whl/cu124
# !pip install --upgrade --quiet torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install torch==2.6.0 torchvision==0.21.0
%cd /content

from IPython.display import clear_output
!git clone https://github.com/Isi-dev/ComfyUI
clear_output()
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/Isi-dev/ComfyUI_GGUF.git
clear_output()
!git clone https://github.com/Isi-dev/ComfyUI_UltimateSDUpscale
clear_output()
# !git clone https://github.com/Isi-dev/ComfyUI_VideoHelperSuite
# clear_output()
%cd /content/ComfyUI/custom_nodes/ComfyUI_GGUF
!pip install -r requirements.txt
clear_output()
# %cd /content/ComfyUI/custom_nodes/ComfyUI_VideoHelperSuite
# !pip install -r requirements.txt
clear_output()
%cd /content/ComfyUI

import subprocess
import sys

def install_pip_packages():
    packages = [
        'torchsde',
        'av',
        'diffusers',
        # 'transformers',
        'xformers==0.0.29.post2',
        'accelerate',
        # 'omegaconf',
        # 'tqdm',
        # 'librosa',
        'einops',
        'spandrel'
    ]

    for package in packages:
        try:
            # Run pip install silently (using -q)
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True,
                capture_output=True
            )
            print(f"✓ {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ Error installing {package}: {e.stderr.decode().strip() or 'Unknown error'}")

def install_apt_packages():
    packages = ['aria2']

    try:
        # Run apt install silently (using -qq)
        subprocess.run(
            ['apt-get', '-y', 'install', '-qq'] + packages,
            check=True,
            capture_output=True
        )
        print("✓ apt packages installed")
    except subprocess.CalledProcessError as e:
        print(f"✗ Error installing apt packages: {e.stderr.decode().strip() or 'Unknown error'}")


print("Installing pip packages...")
install_pip_packages()
clear_output()  # Clear the pip installation output

print("Installing apt packages...")
install_apt_packages()
clear_output()  # Clear the apt installation output

print("Installation completed with status:")
print("- All pip packages installed successfully" if '✗' not in install_pip_packages.__code__.co_consts else "- Some pip packages had issues")
print("- apt packages installed successfully" if '✗' not in install_apt_packages.__code__.co_consts else "- apt packages had issues")

import torch
import numpy as np
from PIL import Image
import gc
import os
import torch
from PIL import Image
import random
import imageio
from google.colab import files
from IPython.display import display, HTML, Image as IPImage
sys.path.insert(0, '/content/ComfyUI')

from nodes import (
    DualCLIPLoader,
    UNETLoader,
    VAELoader,
    LoraLoaderModelOnly,
    LoadImage,
    SaveImage
)

from custom_nodes.ComfyUI_GGUF.nodes import UnetLoaderGGUF

from comfy_extras.nodes_upscale_model import UpscaleModelLoader
from comfy_extras.nodes_flux import CLIPTextEncodeFlux

from custom_nodes.ComfyUI_UltimateSDUpscale.nodes import (
    UltimateSDUpscale,
    UltimateSDUpscaleNoUpscale
)

# from custom_nodes.ComfyUI_VideoHelperSuite.nodes import

from pathlib import Path

download_face_upscalers = False # @param {type:"boolean"}

download_loRA = False # @param {type:"boolean"}

lora = None

def download_with_aria2c(link, folder="/content/ComfyUI/models/loras"):
    import os

    filename = link.split("/")[-1]
    command = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d {folder} -o {filename}"

    print("Executing download command:")
    print(command)

    os.makedirs(folder, exist_ok=True)
    get_ipython().system(command)

    return filename



def download_civitai_model(civitai_link, civitai_token, folder="/content/ComfyUI/models/loras"):
    import os
    import time

    os.makedirs(folder, exist_ok=True)

    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        raise ValueError("Invalid Civitai URL format. Please use a link like: https://civitai.com/api/download/models/1523247?...")

    civitai_url = f"https://civitai.com/api/download/models/{model_id}?type=Model&format=SafeTensor"
    if civitai_token:
        civitai_url += f"&token={civitai_token}"

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f"model_{timestamp}.safetensors"

    full_path = os.path.join(folder, filename)

    download_command = f"wget --max-redirect=10 --show-progress \"{civitai_url}\" -O \"{full_path}\""
    print("Downloading from Civitai...")

    os.system(download_command)

    local_path = os.path.join(folder, filename)
    if os.path.exists(local_path) and os.path.getsize(local_path) > 0:
        print(f"LoRA downloaded successfully: {local_path}")
    else:
        print(f"❌ LoRA download failed or file is empty: {local_path}")

    return filename


def download_lora(link, folder="/content/ComfyUI/models/loras", civitai_token=None):
    """
    Download a model file, automatically detecting if it's a Civitai link or huggingface download.

    Args:
        link: The download URL (either huggingface or Civitai)
        folder: Destination folder for the download
        civitai_token: Optional token for Civitai downloads (required if link is from Civitai)

    Returns:
        The filename of the downloaded model
    """
    if "civitai.com" in link.lower():
        if not civitai_token:
            raise ValueError("Civitai token is required for Civitai downloads")
        return download_civitai_model(link, civitai_token, folder)
    else:
        return download_with_aria2c(link, folder)

flux_lora_download_url = "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/flux_realism_lora.safetensors"# @param {"type":"string"}
token_if_civitai_url = "Put your civitai token here"# @param {"type":"string"}

if download_loRA:
    lora = download_lora(flux_lora_download_url, civitai_token=token_if_civitai_url)
# Validate loRA file extension
valid_extensions = {'.safetensors', '.ckpt', '.pt', '.pth', '.sft'}
if lora:
    if not any(lora.lower().endswith(ext) for ext in valid_extensions):
        print(f"❌ Invalid LoRA format: {lora}")
        lora = None
    else:
        clear_output()
        print("loRA downloaded succesfully!")

def model_download(url: str, dest_dir: str, filename: str = None, silent: bool = True) -> bool:
    """
    Colab-optimized download with aria2c

    Args:
        url: Download URL
        dest_dir: Target directory (will be created if needed)
        filename: Optional output filename (defaults to URL filename)
        silent: If True, suppresses all output (except errors)

    Returns:
        bool: True if successful, False if failed
    """
    try:
        # Create destination directory
        Path(dest_dir).mkdir(parents=True, exist_ok=True)

        # Set filename if not specified
        if filename is None:
            filename = url.split('/')[-1].split('?')[0]  # Remove URL parameters

        # Build command
        cmd = [
            'aria2c',
            '--console-log-level=error',
            '-c', '-x', '16', '-s', '16', '-k', '1M',
            '-d', dest_dir,
            '-o', filename,
            url
        ]

        # Add silent flags if requested
        if silent:
            cmd.extend(['--summary-interval=0', '--quiet'])
            print(f"Downloading {filename}...", end=' ', flush=True)

        # Run download
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)

        if silent:
            print("Done!")
        else:
            print(f"Downloaded {filename} to {dest_dir}")
        return filename

    except subprocess.CalledProcessError as e:
        error = e.stderr.strip() or "Unknown error"
        print(f"\nError downloading {filename}: {error}")
        return False
    except Exception as e:
        print(f"\nError: {str(e)}")
        return False





x_UltraSharp = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/4x-UltraSharp.pth", "/content/ComfyUI/models/upscale_models")
x_foolhardy_Remacri = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/4x_foolhardy_Remacri.pth", "/content/ComfyUI/models/upscale_models")
x_AnimeSharp = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/4x-AnimeSharp.pth", "/content/ComfyUI/models/upscale_models")
if download_face_upscalers:
    x_FaceUpSharpDAT = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/4xFaceUpSharpDAT.pth", "/content/ComfyUI/models/upscale_models")
    x_FaceUpSharpLDAT = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/4xFaceUpSharpLDAT.safetensors", "/content/ComfyUI/models/upscale_models")

flux_model = model_download("https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q8_0.gguf", "/content/ComfyUI/models/unet")
# flux_model = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/flux1-dev-fp8.safetensors", "/content/ComfyUI/models/unet")
flux_vae = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/ae.sft", "/content/ComfyUI/models/vae")
flux_clip_l = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/clip_l.safetensors", "/content/ComfyUI/models/clip")
flux_t5xxl = model_download("https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/t5xxl_fp8_e4m3fn.safetensors", "/content/ComfyUI/models/clip")

clip_loader = DualCLIPLoader()
unet_loader =  UnetLoaderGGUF()
# unet_loader =  UNETLoader()
vae_loader =   VAELoader()
load_lora = LoraLoaderModelOnly()
load_image = LoadImage()
save_image = SaveImage()
upscale_model_loader = UpscaleModelLoader()
positive_prompt_encode = CLIPTextEncodeFlux()
negative_prompt_encode = CLIPTextEncodeFlux()
upscaler = UltimateSDUpscale()
noUpscale = UltimateSDUpscaleNoUpscale()

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    for obj in list(globals().values()):
        if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
            del obj
    gc.collect()


def save_as_image(image, filename_prefix, output_dir="/content/ComfyUI/output"):
    """Save single frame as PNG image."""
    os.makedirs(output_dir, exist_ok=True)
    output_path = f"{output_dir}/{filename_prefix}.png"

    frame = (image.cpu().numpy() * 255).astype(np.uint8)

    Image.fromarray(frame).save(output_path)

    return output_path


# def save_as_mp4(images, filename_prefix, fps, output_dir="/content/ComfyUI/output"):
#     os.makedirs(output_dir, exist_ok=True)
#     output_path = f"{output_dir}/{filename_prefix}.mp4"

#     frames = [(img.cpu().numpy() * 255).astype(np.uint8) for img in images]

#     with imageio.get_writer(output_path, fps=fps) as writer:
#         for frame in frames:
#             writer.append_data(frame)

#     return output_path

def save_as_mp4(images, filename_prefix, fps, output_dir="/content/ComfyUI/output"):
    os.makedirs(output_dir, exist_ok=True)
    output_path = f"{output_dir}/{filename_prefix}.mp4"

    frames = []
    for i, img in enumerate(images):
        try:

            if isinstance(img, torch.Tensor):
                img = img.cpu().numpy()

            # print(f"Frame {i} initial shape: {img.shape}, dtype: {img.dtype}, max: {img.max()}")  # Debug


            if img.max() <= 1.0:
                img = (img * 255).astype(np.uint8)
            else:
                img = img.astype(np.uint8)


            if len(img.shape) == 4:  # Batch dimension? (N, C, H, W)
                img = img[0]  # Take first image in batch

            if len(img.shape) == 3:
                if img.shape[0] in (1, 3, 4):  # CHW format
                    img = np.transpose(img, (1, 2, 0))
                elif img.shape[2] > 4:  # Too many channels
                    img = img[:, :, :3]
            elif len(img.shape) == 2:
                img = np.expand_dims(img, axis=-1)

            # print(f"Frame {i} processed shape: {img.shape}")  # Debug

            # Final validation
            if len(img.shape) != 3 or img.shape[2] not in (1, 3, 4):
                raise ValueError(f"Invalid frame shape after processing: {img.shape}")

            frames.append(img)
        except Exception as e:
            print(f"Error processing frame {i}: {str(e)}")
            raise

    try:
        with imageio.get_writer(output_path, fps=fps) as writer:
            for i, frame in enumerate(frames):
                # print(f"Writing frame {i} with shape: {frame.shape}")  # Debug
                writer.append_data(frame)
    except Exception as e:
        print(f"Error writing video: {str(e)}")
        raise

    return output_path

import cv2
import shutil
from IPython.display import Video
import datetime


def upload_file():
    """Handle file upload (image or video) and return paths."""
    os.makedirs('/content/ComfyUI/input', exist_ok=True)
    uploaded = files.upload()

    paths = []
    for filename in uploaded.keys():
        src_path = f'/content/ComfyUI/{filename}'
        dest_path = f'/content/ComfyUI/input/{filename}'
        shutil.move(src_path, dest_path)
        paths.append(dest_path)
        print(f"File saved to: {dest_path}")

    return paths[0] if paths else None

def extract_frames(video_path, max_frames=None):
    """Extract frames from video and return as a list of images."""
    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frames = []

    while True:
        success, frame = vidcap.read()
        if not success or (max_frames and len(frames) >= max_frames):
            break

        # Convert from BGR to RGB (keeping as numpy array)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    if not frames:
        return None, fps

    print(f"Extracted {len(frames)} frames")
    return frames, fps



def select_every_n_frame(
    frames,
    fps,
    n,
    skip_first=0,
    max_output_frames=0
):

    if not frames or n < 1:
        raise ValueError("Frames must be a non-empty list and n must be >= 1")

    frames_to_use = frames[skip_first:]

    if not frames_to_use:
        print("No frames available after skipping.")
        return [], 0.0

    # Select every nth frame
    selected_frames = frames_to_use[::n]

    # Cap output if needed
    if max_output_frames != 0 and len(selected_frames) > max_output_frames:
        selected_frames = selected_frames[:max_output_frames]

    # Adjust fps
    adjusted_fps = fps / n

    # print(f"Original total frames: {len(frames)}")
    # print(f"Skipped first {skip_first} frames")
    # print(f"Selected every {n}th frame -> {len(selected_frames)} frames")
    if max_output_frames:
        print(f"Frame cap: {max_output_frames} -> Final output: {len(selected_frames)} frames")
    print(f"Adjusted FPS: {adjusted_fps:.2f} -> Final output: {len(selected_frames)} frame")

    return selected_frames, adjusted_fps




def display_video(video_path):
    from IPython.display import HTML
    from base64 import b64encode

    video_data = open(video_path,'rb').read()

    # Determine MIME type based on file extension
    if video_path.lower().endswith('.mp4'):
        mime_type = "video/mp4"
    elif video_path.lower().endswith('.webm'):
        mime_type = "video/webm"
    elif video_path.lower().endswith('.webp'):
        mime_type = "image/webp"
    else:
        mime_type = "video/mp4"  # default

    data_url = f"data:{mime_type};base64," + b64encode(video_data).decode()

    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="{mime_type}">
    </video>
    """))

def upscale_input(
    image_path: str = None,
    positive_prompt: str = "",
    positive_prompt2: str = "",
    negative_prompt: str = "",
    negative_prompt2: str = "",
    guidance: float = 3.5,
    upscale_by: float = 2,
    seed: int = 0,
    steps: int = 20,
    cfg: float = 1.0,
    sampler_name: str = "dpmpp_2m",
    scheduler: str = "karras",
    denoise: float = 0.2,
    upscale_model: str = "4x-UltraSharp.pth",
    mode_type: str = "Linear",
    tile_width: int = 512,
    tile_height: int = 512,
    mask_blur: int = 16,
    tile_padding: int = 32,
    seam_fix_mode: str = "Half Tile",
    seam_fix_denoise: float = 1.0,
    seam_fix_width: int = 64,
    seam_fix_mask_blur: int = 8,
    seam_fix_padding: int = 32,
    force_uniform_tiles: bool = True,
    tiled_decode: bool = False,
    select_every_nth: int = 1,
    skip_first_frames: int = 0,
    max_output_frames: str = None,
    overwrite: bool = False,
    use_lora: bool = False,
    LoRA_Strength: float = 1.0

):

    with torch.inference_mode():

        print("Loading Text_Encoder...")
        clip = clip_loader.load_clip(flux_t5xxl, flux_clip_l, "flux")[0]

        frames=1

        positive = positive_prompt_encode.encode(clip, positive_prompt, positive_prompt2, guidance)[0]
        negative = negative_prompt_encode.encode(clip, negative_prompt, negative_prompt2, guidance)[0]

        del clip
        torch.cuda.empty_cache()
        gc.collect()

        print("Loading Unet Model...")
        model = unet_loader.load_unet(flux_model)[0]
        # model = unet_loader.load_unet(flux_model, "default")[0]

        if image_path is None:
            print("Please upload an image or video file:")
            image_path = upload_file()
        if image_path is None:
            print("No image uploaded!")
        # loaded_image = load_image.load_image(image_path)[0]


        print("Loading upscale model")

        if download_face_upscalers == False:
            if upscale_model == "4xFaceUpSharpDAT.pth" or upscale_model == "4xFaceUpSharpLDAT.safetensors":
                print("You did not download the selected upscale model. Using the default upscale model.")
                upscale_model = "4x-UltraSharp.pth"

        upscale_model_load = upscale_model_loader.load_model(upscale_model)[0]

        print("Loading VAE...")
        vae = vae_loader.load_vae(flux_vae)[0]

        if use_lora:
            if lora is not None:
                print("Loading Lora...")
                model = load_lora.load_lora_model_only(model, lora, LoRA_Strength)[0]

        output_path = ""
        base_name = "Upscaled"
        if not overwrite:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            base_name += f"_{timestamp}"

        try:
            if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Single image processing
                loaded_image = load_image.load_image(image_path)[0]
                print("Upscaling image...")
                image_out = upscaler.upscale(
                    image=loaded_image,
                    model=model,
                    positive=positive,
                    negative=negative,
                    vae=vae,
                    upscale_by=upscale_by,
                    seed=seed,
                    steps=steps,
                    cfg=cfg,
                    sampler_name=sampler_name,
                    scheduler=scheduler,
                    denoise=denoise,
                    upscale_model=upscale_model_load,
                    mode_type=mode_type,
                    tile_width=tile_width,
                    tile_height=tile_height,
                    mask_blur=mask_blur,
                    tile_padding=tile_padding,
                    seam_fix_mode=seam_fix_mode,
                    seam_fix_denoise=seam_fix_denoise,
                    seam_fix_mask_blur=seam_fix_mask_blur,
                    seam_fix_width=seam_fix_width,
                    seam_fix_padding=seam_fix_padding,
                    force_uniform_tiles=force_uniform_tiles,
                    tiled_decode=tiled_decode,
                )[0]



                # output_path = ""
                # base_name = "ComfyUI"
                # if not overwrite:
                #     timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                #     base_name += f"_{timestamp}"
                print("Saving as PNG image...")
                output_path = save_as_image(image_out[0], base_name)
                display(IPImage(filename=output_path))

                del model
                del vae
                del upscale_model_load
                torch.cuda.empty_cache()
                gc.collect()

            else:
                # Video processing
                frames, fps = extract_frames(file_uploaded)
                frames, fps = select_every_n_frame(frames,fps,select_every_nth,skip_first_frames,max_output_frames)
                upscaled_frames = []
                # for i, frame in frames:
                for i, frame in enumerate(frames):
                    temp_path = "/tmp/temp_frame.png"
                    cv2.imwrite(temp_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
                    loaded_image = load_image.load_image(temp_path)[0]

                    print(f"Upscaling frame {i+1} of {len(frames)} frames...")

                    upscaled_frame = upscaler.upscale(
                        image=loaded_image,
                        model=model,
                        positive=positive,
                        negative=negative,
                        vae=vae,
                        upscale_by=upscale_by,
                        seed=seed,
                        steps=steps,
                        cfg=cfg,
                        sampler_name=sampler_name,
                        scheduler=scheduler,
                        denoise=denoise,
                        upscale_model=upscale_model_load,
                        mode_type=mode_type,
                        tile_width=tile_width,
                        tile_height=tile_height,
                        mask_blur=mask_blur,
                        tile_padding=tile_padding,
                        seam_fix_mode=seam_fix_mode,
                        seam_fix_denoise=seam_fix_denoise,
                        seam_fix_mask_blur=seam_fix_mask_blur,
                        seam_fix_width=seam_fix_width,
                        seam_fix_padding=seam_fix_padding,
                        force_uniform_tiles=force_uniform_tiles,
                        tiled_decode=tiled_decode
                    )[0]
                    upscaled_frames.append(upscaled_frame)



                print(f"Saving as MP4 with {len(upscaled_frames)} frames...")
                output_path = save_as_mp4(upscaled_frames, base_name, fps)
                display_video(output_path)

                del model
                del vae
                del upscale_model_load
                torch.cuda.empty_cache()
                gc.collect()

        except Exception as e:
            print(f"Error during upscaling/saving: {str(e)}")
            raise
        finally:
            clear_memory()


print("✅ Environment Setup Complete!")

In [ ]:
# @title Upload Image/Video

file_uploaded = upload_file()
display_upload = False # @param {type:"boolean"}
if display_upload:
    if file_uploaded.lower().endswith(('.png', '.jpg', '.jpeg')):
        display(IPImage(filename=file_uploaded))
    else:
        display_video(file_uploaded)

In [ ]:
# @title Upscale Image/Video

# @markdown ---
# @markdown ### Prompt Settings
positive_prompt="" # @param {"type":"string"}
positive_prompt2="" # @param {"type":"string"}
negative_prompt="" # @param {"type":"string"}
negative_prompt2="" # @param {"type":"string"}
guidance=3.5 # @param {"type":"slider","min":0.0,"max":100.0,"step":0.1}
# @markdown ---
# @markdown ### Upscale Settings
upscale_by=2 # @param {"type":"slider","min":0.05,"max":4.0,"step":0.05}
seed=0 # @param {"type":"integer"}
steps = 20 # @param {"type":"slider","min":0,"max":1000,"step":1}
cfg = 7 # @param {"type":"slider","min":0,"max":20,"step":1}
sampler_name="euler" # @param ["uni_pc", "uni_pc_bh2", "ddim","euler", "euler_cfg_pp", "euler_ancestral", "euler_ancestral_cfg_pp", "heun", "heunpp2","dpm_2", "dpm_2_ancestral","lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_2s_ancestral_cfg_pp", "dpmpp_sde", "dpmpp_sde_gpu","dpmpp_2m", "dpmpp_2m_cfg_pp", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "dpmpp_3m_sde", "dpmpp_3m_sde_gpu", "ddpm", "lcm","ipndm", "ipndm_v", "deis", "res_multistep", "res_multistep_cfg_pp", "res_multistep_ancestral", "res_multistep_ancestral_cfg_pp","gradient_estimation", "er_sde", "seeds_2", "seeds_3"]
scheduler="normal" # @param ["simple","normal","karras","exponential","sgm_uniform","ddim_uniform","beta","linear_quadratic","kl_optimal"]
denoise=0.2 # @param {"type":"slider","min":0.0,"max":1.0,"step":0.01}
upscale_model="4x-UltraSharp.pth" # @param ["4x-UltraSharp.pth", "4x_foolhardy_Remacri.pth", "4x-AnimeSharp.pth", "4xFaceUpSharpDAT.pth", "4xFaceUpSharpLDAT.safetensors"]
mode_type="Linear" # @param ["Linear", "Chess", "None"]
tile_width = 512 # @param {"type":"slider","min":64,"max":8192,"step":8}
tile_height = 512 # @param {"type":"slider","min":64,"max":8192,"step":8}
mask_blur = 8 # @param {"type":"slider","min":0,"max":64,"step":1}
tile_padding = 32 # @param {"type":"slider","min":0,"max":8192,"step":8}
seam_fix_mode="None" # @param ["None", "Half Tile", "Half Tile + Intersections", "Band Pass"]
seam_fix_denoise=1.0 # @param {"type":"slider","min":0.0,"max":1.0,"step":0.01}
seam_fix_width=64 # @param {"type":"slider","min":0,"max":8192,"step":8}
seam_fix_mask_blur=8 # @param {"type":"slider","min":0,"max":64,"step":1}
seam_fix_padding=16 # @param {"type":"slider","min":0,"max":8192,"step":8}
force_uniform_tiles=True # @param {type:"boolean"}
tiled_decode=False # @param {type:"boolean"}
# @markdown ---
# @markdown ### LoRA Settings
use_loRA=False # @param {type:"boolean"}
LoRA_Strength=1.0 # @param {"type":"slider","min":-100,"max":100,"step":0.01}
# @markdown ---
# @markdown ### Video Settings
select_every_nth_frame = 1 # @param {"type":"integer"}
skip_first_frames = 0 # @param {"type":"integer"}
max_output_frames = 0 # @param {"type":"integer"}
overwrite_previous_output=False # @param {type:"boolean"}


import random
seed = seed if seed != 0 else random.randint(0, 2**32 - 1)
print(f"Using seed: {seed}")

upscale_input(
    image_path=file_uploaded,
    positive_prompt=positive_prompt,
    positive_prompt2=positive_prompt2,
    negative_prompt=negative_prompt,
    negative_prompt2=negative_prompt2,
    guidance=guidance,
    upscale_by=upscale_by,
    seed=seed,
    steps=steps,
    cfg=cfg,
    sampler_name=sampler_name,
    scheduler=scheduler,
    denoise=denoise,
    upscale_model=upscale_model,
    mode_type=mode_type,
    tile_width=tile_width,
    tile_height=tile_height,
    mask_blur=mask_blur,
    tile_padding=tile_padding,
    seam_fix_mode=seam_fix_mode,
    seam_fix_denoise=seam_fix_denoise,
    seam_fix_width=seam_fix_width,
    seam_fix_mask_blur=seam_fix_mask_blur,
    seam_fix_padding=seam_fix_padding,
    force_uniform_tiles=force_uniform_tiles,
    tiled_decode=tiled_decode,
    select_every_nth = select_every_nth_frame,
    skip_first_frames = skip_first_frames,
    max_output_frames = max_output_frames,
    overwrite=overwrite_previous_output,
    use_lora=use_loRA,
    LoRA_Strength=LoRA_Strength

)

clear_memory()